# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 4 2022 2:36PM,247879,10,8610054,Yusen – 3D Matrix,Released
1,Oct 4 2022 2:34PM,247878,19,ADV80006177,"AdvaGen Pharma, Ltd",Released
2,Oct 4 2022 2:34PM,247878,19,ADV80006178,"AdvaGen Pharma, Ltd",Released
3,Oct 4 2022 2:34PM,247878,19,ADV80006179,"AdvaGen Pharma, Ltd",Released
4,Oct 4 2022 2:34PM,247878,19,ADV80006180,"AdvaGen Pharma, Ltd",Released
5,Oct 4 2022 2:34PM,247878,19,ADV80006181,"AdvaGen Pharma, Ltd",Released
6,Oct 4 2022 2:34PM,247878,19,ADV80006182,"AdvaGen Pharma, Ltd",Released
7,Oct 4 2022 2:34PM,247878,19,ADV80006183,"AdvaGen Pharma, Ltd",Released
8,Oct 4 2022 2:34PM,247878,19,ADV80006184,"AdvaGen Pharma, Ltd",Released
9,Oct 4 2022 2:34PM,247878,19,ADV80006185,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,247872,Released,1
15,247875,Released,1
16,247876,Released,1
17,247878,Released,21
18,247879,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247872,NaN,NaN,1.0
247875,NaN,NaN,1.0
247876,NaN,NaN,1.0
247878,NaN,NaN,21.0
247879,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247794,0.0,0.0,1.0
247796,0.0,6.0,2.0
247811,3.0,5.0,22.0
247819,0.0,0.0,1.0
247829,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
247794,0,0,1
247796,0,6,2
247811,3,5,22
247819,0,0,1
247829,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247794,0,0,1
1,247796,0,6,2
2,247811,3,5,22
3,247819,0,0,1
4,247829,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,247794,,,1
1,247796,,6,2
2,247811,3,5,22
3,247819,,,1
4,247829,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 4 2022 2:36PM,247879,10,Yusen – 3D Matrix
1,Oct 4 2022 2:34PM,247878,19,"AdvaGen Pharma, Ltd"
22,Oct 4 2022 2:12PM,247876,19,Paragon BioTeck
23,Oct 4 2022 2:09PM,247875,20,"ACI Healthcare USA, Inc."
24,Oct 4 2022 1:38PM,247872,19,"Granules Pharmaceuticals, Inc."
25,Oct 4 2022 1:34PM,247871,10,"Nextsource Biotechnology, LLC"
30,Oct 4 2022 12:59PM,247864,10,Eywa Pharma Inc.
49,Oct 4 2022 12:41PM,247856,10,"Methapharm, Inc."
94,Oct 4 2022 12:32PM,247853,10,MedStone Pharma LLC
100,Oct 4 2022 11:33AM,247839,18,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 4 2022 2:36PM,247879,10,Yusen – 3D Matrix,,,1
1,Oct 4 2022 2:34PM,247878,19,"AdvaGen Pharma, Ltd",,,21
2,Oct 4 2022 2:12PM,247876,19,Paragon BioTeck,,,1
3,Oct 4 2022 2:09PM,247875,20,"ACI Healthcare USA, Inc.",,,1
4,Oct 4 2022 1:38PM,247872,19,"Granules Pharmaceuticals, Inc.",,,1
5,Oct 4 2022 1:34PM,247871,10,"Nextsource Biotechnology, LLC",,2,3
6,Oct 4 2022 12:59PM,247864,10,Eywa Pharma Inc.,,,19
7,Oct 4 2022 12:41PM,247856,10,"Methapharm, Inc.",,,45
8,Oct 4 2022 12:32PM,247853,10,MedStone Pharma LLC,,,6
9,Oct 4 2022 11:33AM,247839,18,"Emersa Waterbox, LLC",,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 4 2022 2:36PM,247879,10,Yusen – 3D Matrix,1,,
1,Oct 4 2022 2:34PM,247878,19,"AdvaGen Pharma, Ltd",21,,
2,Oct 4 2022 2:12PM,247876,19,Paragon BioTeck,1,,
3,Oct 4 2022 2:09PM,247875,20,"ACI Healthcare USA, Inc.",1,,
4,Oct 4 2022 1:38PM,247872,19,"Granules Pharmaceuticals, Inc.",1,,
5,Oct 4 2022 1:34PM,247871,10,"Nextsource Biotechnology, LLC",3,2,
6,Oct 4 2022 12:59PM,247864,10,Eywa Pharma Inc.,19,,
7,Oct 4 2022 12:41PM,247856,10,"Methapharm, Inc.",45,,
8,Oct 4 2022 12:32PM,247853,10,MedStone Pharma LLC,6,,
9,Oct 4 2022 11:33AM,247839,18,"Emersa Waterbox, LLC",,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 4 2022 2:36PM,247879,10,Yusen – 3D Matrix,1,,
1,Oct 4 2022 2:34PM,247878,19,"AdvaGen Pharma, Ltd",21,,
2,Oct 4 2022 2:12PM,247876,19,Paragon BioTeck,1,,
3,Oct 4 2022 2:09PM,247875,20,"ACI Healthcare USA, Inc.",1,,
4,Oct 4 2022 1:38PM,247872,19,"Granules Pharmaceuticals, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 4 2022 2:36PM,247879,10,Yusen – 3D Matrix,1.0,NaN,NaN
1,Oct 4 2022 2:34PM,247878,19,"AdvaGen Pharma, Ltd",21.0,NaN,NaN
2,Oct 4 2022 2:12PM,247876,19,Paragon BioTeck,1.0,NaN,NaN
3,Oct 4 2022 2:09PM,247875,20,"ACI Healthcare USA, Inc.",1.0,NaN,NaN
4,Oct 4 2022 1:38PM,247872,19,"Granules Pharmaceuticals, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 4 2022 2:36PM,247879,10,Yusen – 3D Matrix,1.0,0.0,0.0
1,Oct 4 2022 2:34PM,247878,19,"AdvaGen Pharma, Ltd",21.0,0.0,0.0
2,Oct 4 2022 2:12PM,247876,19,Paragon BioTeck,1.0,0.0,0.0
3,Oct 4 2022 2:09PM,247875,20,"ACI Healthcare USA, Inc.",1.0,0.0,0.0
4,Oct 4 2022 1:38PM,247872,19,"Granules Pharmaceuticals, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1982747,98.0,7.0,3.0
16,247796,2.0,6.0,0.0
18,247839,0.0,1.0,0.0
19,743626,23.0,0.0,0.0
20,495704,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1982747,98.0,7.0,3.0
1,16,247796,2.0,6.0,0.0
2,18,247839,0.0,1.0,0.0
3,19,743626,23.0,0.0,0.0
4,20,495704,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,98.0,7.0,3.0
1,16,2.0,6.0,0.0
2,18,0.0,1.0,0.0
3,19,23.0,0.0,0.0
4,20,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,98.0
1,16,Released,2.0
2,18,Released,0.0
3,19,Released,23.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,18,19,20
Status,,,,,
Completed,3.0,0.0,0.0,0.0,0.0
Executing,7.0,6.0,1.0,0.0,1.0
Released,98.0,2.0,0.0,23.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,18,19,20
0,Completed,3.0,0.0,0.0,0.0,0.0
1,Executing,7.0,6.0,1.0,0.0,1.0
2,Released,98.0,2.0,0.0,23.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,18,19,20
0,Completed,3.0,0.0,0.0,0.0,0.0
1,Executing,7.0,6.0,1.0,0.0,1.0
2,Released,98.0,2.0,0.0,23.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()